In [1]:
from acevedo_clss_and_fcns import * 
device = 'cpu'
if torch.cuda.is_available():
    torch.cuda.init()
    if torch.cuda.is_initialized():
        device = 'cuda:0'
print(f"{device = }")



/DeepenData/.miniconda/envs/geo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device = 'cuda:0'


In [9]:
def train_one_epoch(modelo: GIN_classifier_to_explain,
                    optimizer, 
                    train_loader: torch_geometric.loader.dataloader.DataLoader,
                    loss_fun: torch.nn.modules.loss,
                    device:str='cpu' ):

    correct = 0
    for i, data in enumerate(train_loader):
        assert not data.is_cuda   
        if (device == 'cuda:0') | (device == 'cuda'):                            
            data.to(device, non_blocking=True) 
            assert data.is_cuda       
                
        optimizer.zero_grad(set_to_none=True) # Zero your gradients for every batch        
        if (device == 'cuda:0') | (device == 'cuda'):
            #with torch.cuda.amp.autocast():      
            predictions = modelo(data.x, data.edge_index,  data.batch)# Make predictions for this batch
            loss        = loss_fun(predictions, data.y)
            loss.backward()  # Derive gradients.
            optimizer.step()  # Update parameters based on gradients.        
            pred     = predictions.argmax(dim=1)  # Use the class with highest probability.
            correct += int((pred == data.y).sum())  # Check against ground-truth labels.

    return correct / len(train_loader.dataset)

def validate(modelo: GIN_classifier_to_explain, loader: DataLoader, device: str = 'cpu'):
    modelo.eval()
    correct = 0
    for i, val_data in enumerate(loader):
        
        assert not val_data.is_cuda
        if (device == 'cuda:0') | (device == 'cuda'):
            val_data.to(device, non_blocking=True) 
            assert val_data.is_cuda                          

        val_predictions = modelo(val_data.x, val_data.edge_index,  val_data.batch)# Make predictions for this batch
        pred            = val_predictions.argmax(dim=1)

        correct += int((pred == val_data.y).sum())
        

    return correct / len(loader.dataset)   

#loader_path = "./results/dataloaders/MASKED_loader_Concen_plus_Fluxes.pt"
saving_path     = "./results/trained_pytorch_models"
model_subfolder = "/Non_masked_Phe"
loader_path     = "./results/dataloaders/loader_Concen_plus_Fluxes.pt"
loader = torch.load(loader_path)
a_batch         = next(iter(loader.get_train_loader()))
a_graph         = a_batch[0]
model           = GIN_classifier_to_explain_v2(
                                            n_nodes = a_graph.num_nodes, 
                                            num_features = a_graph.num_node_features, 
                                            n_classes = a_graph.num_classes,
                                            hidden_dim=8,
                                            num_layers=2).to(device, non_blocking=True).to(device)


optimizer       = torch.optim.Adam(model.parameters())
loss_function   = torch.nn.NLLLoss()
best_validation_accuracy = 1e-10
EPOCHS = 10
verbose = True


In [10]:
gc.collect()
torch.cuda.empty_cache() 
for epoch in tqdm.tqdm(range(EPOCHS)):
    
    train_accuracy = train_one_epoch(model,
                        optimizer=optimizer, 
                        train_loader=loader.get_train_loader(),
                        loss_fun=loss_function,
                        device = device)

    validation_accuracy = validate(model, loader.get_validation_loader(), device)
    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        del validation_accuracy
        best_val_state_dict   = copy.deepcopy(model.state_dict())
        best_val_model        = copy.deepcopy(model)
        if verbose:
            timestamp     = datetime.now().strftime('%d-%m-%Y_%Hh_%Mmin')              
            print(f'Epoch: {epoch:03d}, train_accuracy: {train_accuracy:.4f}, best_validation_accuracy: {best_validation_accuracy:.4f}')
            model_path = saving_path+model_subfolder+'/Model_{}_{}_best_ValAcc_{}_epoch_{}.pt'.format(model.__class__.__name__,timestamp, best_validation_accuracy, epoch)
            torch.save(best_val_model, model_path)
            print(f"saved as {model_path}")



 10%|█         | 1/10 [00:03<00:34,  3.82s/it]

Epoch: 000, train_accuracy: 0.7494, best_validation_accuracy: 0.9619
saved as ./results/trained_pytorch_models/Non_masked_Phe/Model_GIN_classifier_to_explain_v2_23-10-2022_15h_00min_best_ValAcc_0.961901169370049_epoch_0.pt


 20%|██        | 2/10 [00:07<00:30,  3.77s/it]

Epoch: 001, train_accuracy: 0.9774, best_validation_accuracy: 0.9800
saved as ./results/trained_pytorch_models/Non_masked_Phe/Model_GIN_classifier_to_explain_v2_23-10-2022_15h_00min_best_ValAcc_0.980007544322897_epoch_1.pt


 40%|████      | 4/10 [00:15<00:22,  3.76s/it]

...

 60%|██████    | 6/10 [00:22<00:15,  3.76s/it]

Epoch: 005, train_accuracy: 0.9872, best_validation_accuracy: 0.9887
saved as ./results/trained_pytorch_models/Non_masked_Phe/Model_GIN_classifier_to_explain_v2_23-10-2022_15h_00min_best_ValAcc_0.98868351565447_epoch_5.pt


 70%|███████   | 7/10 [00:26<00:11,  3.75s/it]

Epoch: 006, train_accuracy: 0.9862, best_validation_accuracy: 0.9891
saved as ./results/trained_pytorch_models/Non_masked_Phe/Model_GIN_classifier_to_explain_v2_23-10-2022_15h_00min_best_ValAcc_0.989060731799321_epoch_6.pt


100%|██████████| 10/10 [00:37<00:00,  3.75s/it]


In [15]:
saving_path     = "./results/trained_pytorch_models"
model_subfolder = "/Masked_Phe"
loader_path     = "./results/dataloaders/MASKED_loader_Concen_plus_Fluxes.pt"
loader = torch.load(loader_path)
model           = GIN_classifier_to_explain_v2(
                                            n_nodes = a_graph.num_nodes, 
                                            num_features = a_graph.num_node_features, 
                                            n_classes = a_graph.num_classes,
                                            hidden_dim=8,
                                            num_layers=2).to(device, non_blocking=True).to(device)

optimizer       = torch.optim.Adam(model.parameters())
loss_function   = torch.nn.NLLLoss()
best_validation_accuracy = 1e-10
EPOCHS = 50
verbose = True
gc.collect()
torch.cuda.empty_cache() 
for epoch in tqdm.tqdm(range(EPOCHS)):
    
    train_accuracy = train_one_epoch(model,
                        optimizer=optimizer, 
                        train_loader=loader.get_train_loader(),
                        loss_fun=loss_function,
                        device = device)

    validation_accuracy = validate(model, loader.get_validation_loader(), device)
    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        del validation_accuracy
        best_val_state_dict   = copy.deepcopy(model.state_dict())
        best_val_model        = copy.deepcopy(model)
        if verbose:
            timestamp     = datetime.now().strftime('%d-%m-%Y_%Hh_%Mmin')              
            print(f'Epoch: {epoch:03d}, train_accuracy: {train_accuracy:.4f}, best_validation_accuracy: {best_validation_accuracy:.4f}')
            model_path = saving_path+model_subfolder+'/Model_{}_{}_best_ValAcc_{}_epoch_{}.pt'.format(model.__class__.__name__,timestamp, best_validation_accuracy, epoch)
            torch.save(best_val_model, model_path)
            print(f"saved as {model_path}")

  2%|▏         | 1/50 [00:03<03:04,  3.77s/it]

Epoch: 000, train_accuracy: 0.5097, best_validation_accuracy: 0.4855
saved as ./results/trained_pytorch_models/Masked_Phe/Model_GIN_classifier_to_explain_v2_23-10-2022_15h_06min_best_ValAcc_0.4854771784232365_epoch_0.pt


  4%|▍         | 2/50 [00:07<03:00,  3.76s/it]

Epoch: 001, train_accuracy: 0.6570, best_validation_accuracy: 0.7292
saved as ./results/trained_pytorch_models/Masked_Phe/Model_GIN_classifier_to_explain_v2_23-10-2022_15h_06min_best_ValAcc_0.7291588079969823_epoch_1.pt


  6%|▌         | 3/50 [00:11<02:55,  3.74s/it]

...

Epoch: 039, train_accuracy: 0.9075, best_validation_accuracy: 0.9095
saved as ./results/trained_pytorch_models/Masked_Phe/Model_GIN_classifier_to_explain_v2_23-10-2022_15h_08min_best_ValAcc_0.90946812523576_epoch_39.pt


 94%|█████████▍| 47/50 [02:56<00:11,  3.76s/it]

Epoch: 046, train_accuracy: 0.9132, best_validation_accuracy: 0.9121
saved as ./results/trained_pytorch_models/Masked_Phe/Model_GIN_classifier_to_explain_v2_23-10-2022_15h_09min_best_ValAcc_0.9121086382497171_epoch_46.pt


100%|██████████| 50/50 [03:07<00:00,  3.76s/it]

Epoch: 049, train_accuracy: 0.9179, best_validation_accuracy: 0.9132
saved as ./results/trained_pytorch_models/Masked_Phe/Model_GIN_classifier_to_explain_v2_23-10-2022_15h_09min_best_ValAcc_0.91324028668427_epoch_49.pt


# -------------end-----------------